In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


##The Eigenvalues of the Laplacian

In this notebook we will find out how to get the eigenvalues of the Laplacian. We will do this by using the QR method. I used two sources for implementing the QR method, and the two methods are slightly different, but naturally give the same result. The first method is based off of the method presented in Hoffman's Numerical Analysis Book.

### Hoffman's QR Method


$$A = \left[ \begin{matrix} a_{11} ~ a_{12}~ ... ~a_{1n}\\ a_{21} ~a_{22} ~... ~a_{2n}\\ . ~~ . ~~ .\\ a_{n1}  ~ a_{n2} ~ ... ~ a_{nn} \end{matrix} \right]= [\mathbf{a}_1 ~~ \mathbf{a}_2 ~ . ~ . ~ .  ~~  \mathbf{a}_n]$$


Use the Gram-Schmidt process:

$$\mathbf{q}_1= \frac{\mathbf{a}_1} {||\mathbf{a}_1||}$$

$$\mathbf{a}'_2 = \mathbf{a}_2 - (\mathbf{q}_1^T \mathbf{a}_2) \mathbf{q}_1$$

$$\mathbf{q}_2= \frac{\mathbf{a}'_2} {||\mathbf{a}'_2||}$$

$$\mathbf{a}'_3 = \mathbf{a}_3 - (\mathbf{q}_1^T \mathbf{a}_3) \mathbf{q}_1 - (\mathbf{q}_2^T \mathbf{a}_3) \mathbf{q}_2$$

$$\mathbf{q}_3= \frac{\mathbf{a}'_3} {||\mathbf{a}'_3||}$$

and so on...

The matrix <b>Q</b> is composed of the column vectors

The matrix <b>R</b> is assembled from the elements computed in the evaluation of <b>Q</b>

$$r_{i,i} = ||\mathbf{a}'_i||$$

$$r_{i,j} = \mathbf{q}_i^T \mathbf{a}_j ~~~~~~ \text{where} ~~~~ j=i+1, ..., n$$

The first step in the QR process is to set $\mathbf{A}^{(0)} = \mathbf{A}$ and factor $\mathbf{A}^{(0)}$ by the Gram-Schmidt process into $\mathbf{Q}^{(0)}$ and $\mathbf{R}^{(0)}$. The next step is to reverse the factors  $\mathbf{Q}^{(0)}$ and $\mathbf{R}^{(0)}$ to obtain

$$\mathbf{A}^{(1)} =  \mathbf{R}^{(0)} \mathbf{Q}^{(0)} $$

$\mathbf{A}^{(1)}$ is similar to $\mathbf{A}$, so the eigenvalues are preserved. $\mathbf{A}^{(1)}$ is factored by the Gram-Schmidt process to obtain $\mathbf{Q}^{(1)}$ and $\mathbf{R}^{(1)}$, and the factors are reversed to obtain $\mathbf{A}^{(2)}$. Thus,

$$\mathbf{A}^{(2)} =  \mathbf{R}^{(1)} \mathbf{Q}^{(1)} $$

The process is continued to determine $\mathbf{A}^{(3)}, \mathbf{A}^{(4)}, ... , \mathbf{A}^{(n)}$. When $\mathbf{A}^{(n)}$ approaches triangular form, within some tolerance, the eigenvalues of $\mathbf{A}$ are the diagonal elements.

In [5]:
def hoffman_QR(a):
    A = a.astype(float)
    n = A.shape[0]
    q = zeros((n,n))
    r = zeros((n,n))
    a_prime = zeros(n)
    for k in range(20):
        for i in range(n):
            a_prime = A[:,i]
            for j in range(i):
                r[j][i] = dot(q[:,j], A[:,i])
                a_prime -= r[j][i]*q[:,j]
            r[i][i] = norm(a_prime)    
            q[:,i] = a_prime/r[i][i]  
        A = dot(r,q)
    return array([around(A[i][i],8) for i in range(n)])
    
    
A = array([[2,-1,-1,0],[-1,2,0,-1],[-1,0,2,-1],[0,-1,-1,2]])    
hoffman_QR(A)   

array([ 4.,  2.,  2., -0.])

###Tamu's method

The next method I found is based on Householder's matrices and is implemented based on my readings from: http://www.math.tamu.edu/~dallen/linear_algebra/chpt6.pdf


"The basis for the QR method for calculating the eigenvalues of A is the fact that an $n \times n$ real matrix can be written as 

$$A = QR$$

where Q is orthogonal and R is upper triangular. (I'll probably need some of these definitions as well later). The method is efficient for the calculation of all eigenvalues of a matrix."

"The construction of Q and R proceeds as follows. Matrices $P_1, P_2, ... , P_{n-1}$ are constructed so that $P_{n-1}P_{n-2}\cdot \cdot \cdot P_2 P_1 A = R$ is upper triangular. These matrices can be chosen as orthogonal matrices and are called <i>householder matrices</i>. If we let 

$$Q^T = P_{n-1}P_{n-2}\cdot \cdot \cdot P_2 P_1$$

then we have $Q^T A = R$" and therefore $A = QR$ (because of orthogonality or something like that)

"We discuss the construction of the P's presently. First we state how the QR factorization of A is used to find eigenvalues of A. We define sequences of matrices $A_1, A_2, ..., A_m, ...; Q_1, Q_2, ..., Q_m, ... ; R_1, R_2, ..., R_m, ...$ by this process:

<b>Step 1.</b> Set $A_1 = A, Q_1 = Q$ and $R_1 = R$

<b>Step 2.</b> Set $A_2 = R_{1} Q_{1}$; then factor $A_2$ as $A_2 = Q_2 R_2$ (QR factorization of $A_2$)

<b>Step m.</b> Set $A_m = R_{m-1} Q_{m-1}$; then factor $A_m$ as $A_m = Q_m R_m$ (QR factorization of $A_m$)


"Matrix $A_m$ will tend toward a triangular or nearly triangular  form. Thus the eigenvalues will of $A_m$ will be easy to calculate",i.e., they're just the diagonal entries ". The importance is that if the eigenvalues can be ordered as $|\lambda_1|>|\lambda_2|> ... > |\lambda_n|>0$, then the following is true:

As m increases the eigenvalues of $A_m$ approach the eigenvalues of A.

The proof of this fact is well beyond the scope of this book.

Furthermore, "If A is symmetric, matrices $A_m$ converge to a diagonal matrix with the eigenvalues on the diagonal"

"Finally, after we find the eigenvalues of A, the corresponding eigenvectors can be found by solving $(\lambda \mathbb{I} - A) X = 0$, subject to some side condition such as $|X| = 1$"





"The idea in QR factorization is to first find $P_1$ which, when multiplied on the left of A, will produce zeros belos $a_{11}$. That is, we want

$$P_1 \left(\begin{matrix} 
a_{11} \ a_{12} \ ... \ a_{1n} \\
a_{21} \ a_{22} \ ... \ a_{2n} \\
\vdots \ ~~~ \ddots \ ~~~ \vdots \\
a_{n1} \ a_{n2} \ ... \ a_{nn}\\
\end{matrix}\right)
=
 \left(\begin{array} 
\tilde {a_{11}} \ ~~ \tilde {a_{12}} \ ... \ \tilde {a_{1n}} \\
\textbf{0} \ ~~ \tilde {a_{22}} \ ... \ \tilde {a_{2n}} \\
\vdots \ ~~~~~ \ddots \ ~~~~~ \vdots \\
\textbf{0} \ ~~ \tilde {a_{n2}} \ ... \ \tilde {a_{nn}}\\
\end{array}\right)
$$

After this is done, we find $P_2$ which will produce


$$P_2 P_1 A = P_2  \left(\begin{matrix} 
\tilde {a_{11}} \ ~~ \tilde {a_{12}} \ ... \ \tilde {a_{1n}} \\
\textbf{0} \ ~~ \tilde {a_{22}} \ ... \ \tilde {a_{2n}} \\
\vdots \ ~~~~~ \ddots \ ~~~~~ \vdots \\
\textbf{0} \ ~~ \tilde {a_{n2}} \ ... \ \tilde {a_{nn}}\\
\end{matrix}\right)=
 \left(\begin{matrix} 
\hat {a_{11}} \ ~ \hat {a_{12}} ~ \hat {a_{13}} \ ... \ \hat {a_{1n}} \\
\textbf{0} \ ~ \hat {a_{22}} ~ \hat {a_{23}} \ ... \ \hat {a_{2n}} \\
\textbf{0} \ ~~ \textbf {0} ~~~~~ \hat {a_{33}}  \ ... \ ~~ \hat {a_{3n}} \\
\vdots \ ~~~~ \ddots \ ~~~~~ \vdots \\
\textbf{0} \ ~~~~~ \textbf{0} \ ~~ \hat {a_{3n}} \ ... \ \hat {a_{nn}}\\
\end{matrix}\right)
$$

And we do this repeatedly until convergence:

In [8]:
def tamu_QR(a):
    A = a.astype(float)
    n = len(A)
    for j in range(20):
        for k in range(n-1):
            a = A[:,k]
            lower_norm = sqrt(sum([a[i]**2 for i in range(k,n)]))
            alpha = - (sign(a[k])) * lower_norm
            r = sqrt(0.5*(alpha**2 - a[k]*alpha))
            w = zeros(n)
            w[k] = (a[k] - alpha)/(2*r)
            for i in range(k+1,n):
                w[i] = a[i]/(2*r)
            p = eye(n) - 2*outer(w,w.T)
            R = dot(p,A)
            A = dot(R,p)
  
    return array([A[i][i] for i in range(n)])
    
    
A = array([[2,-1,-1,0],[-1,2,0,-1],[-1,0,2,-1],[0,-1,-1,2]]) 
tamu_QR(A)  

array([  4.00000000e+00,   2.00000000e+00,   2.00000000e+00,
         3.52765928e-17])